In [194]:
import pandas as pd
import linecache
from datetime import datetime as dt
import numpy as np

In [195]:
file = '74-2837441_KansasHeartHospital_standardcharges.csv'

In [196]:
date_updated = linecache.getline(file, 3). split(": ")[-1].replace('"\n', '')
date_updated = dt.strptime(date_updated, '%m/%d/%Y').strftime('%Y-%m-%d')

In [197]:
df = pd.read_csv(file, skiprows=3, nrows=6000, dtype={"CPT_CODE": str, "REV_CODE": str})

In [198]:
df = df.rename(columns={
    'PRIMARY_PROCEDURE_DESCRIPTION': 'description',
    'CPT_CODE': 'code',    
})

df.columns = df.columns.str.lower()

df = df.drop(columns=['setting_type', 'service_component_breakout_type', 'service_component_breakout_name', 'drg_code'])

In [ ]:
df['hospital_tin'] = '74-2837441'
df['hospital_ccn'] = '170186'
df['url'] = 'https://assets.changehealthcare.com/Shop/PROD/static/KansasHeartHospital/ein_KansasHeartHospital_standardcharges.csv.zip'


In [199]:
# Set code_prefix to 'cpt' if CPT_CODE is not NA
df.loc[df['code'].notna(), 'code_prefix'] = 'cpt'
df['code'] = df['code'].fillna(df['hcpcs_code'])
df['code_prefix'] = 'hcpcs_cpt'

df.drop(columns=['hcpcs_code'], inplace=True)
df.dropna(subset=['code'], inplace=True)

In [200]:
df = df.head(2)

In [207]:
df_insurances = df.drop(columns = ['gross_price', 'cash_price', 'deidentified_min_price', 'deidentified_max_price'])
df_insurances.rename(columns={'negotiated_price': 'rate'}, inplace=True)

In [208]:
df_insurances

,description,code,modifier,rev_code,rate,payer_name,network_name,code_prefix
755,PHARMACY,B5200,NaN,0258,35.91,Aetna,Commercial,hcpcs_cpt
775,PHARMACY,B5200,NaN,0258,51.30,United Healthcare,Commercial,hcpcs_cpt


In [201]:
payer_vars = ['gross_price', 'cash_price', 'deidentified_min_price', 'deidentified_max_price']
id_vars = [x for x in df.columns if x not in payer_vars]
# id_vars.append('payer_name')
df1 = pd.melt(df, id_vars=id_vars, value_vars=payer_vars, var_name='payer_orig', value_name='rate')


In [202]:
df1

,description,code,modifier,rev_code,negotiated_price,payer_name,network_name,code_prefix,payer_orig,rate
0,PHARMACY,B5200,NaN,0258,35.91,Aetna,Commercial,hcpcs_cpt,gross_price,102.60
1,PHARMACY,B5200,NaN,0258,51.30,United Healthcare,Commercial,hcpcs_cpt,gross_price,102.60
2,PHARMACY,B5200,NaN,0258,35.91,Aetna,Commercial,hcpcs_cpt,cash_price,102.60
3,PHARMACY,B5200,NaN,0258,51.30,United Healthcare,Commercial,hcpcs_cpt,cash_price,102.60
4,PHARMACY,B5200,NaN,0258,35.91,Aetna,Commercial,hcpcs_cpt,deidentified_min_price,35.91
5,PHARMACY,B5200,NaN,0258,51.30,United Healthcare,Commercial,hcpcs_cpt,deidentified_min_price,35.91
6,PHARMACY,B5200,NaN,0258,35.91,Aetna,Commercial,hcpcs_cpt,deidentified_max_price,51.30
7,PHARMACY,B5200,NaN,0258,51.30,United Healthcare,Commercial,hcpcs_cpt,deidentified_max_price,51.30


In [203]:
df

,description,code,modifier,rev_code,gross_price,cash_price,negotiated_price,deidentified_min_price,deidentified_max_price,payer_name,network_name,code_prefix
755,PHARMACY,B5200,NaN,0258,102.6,102.6,35.91,35.91,51.3,Aetna,Commercial,hcpcs_cpt
775,PHARMACY,B5200,NaN,0258,102.6,102.6,51.30,35.91,51.3,United Healthcare,Commercial,hcpcs_cpt
